<a href="https://colab.research.google.com/github/hwarang97/Image_classification/blob/main/VGG16_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG
- 16, 19 등 뒤에 붙는 숫자는 구성된 층의 갯수
- 쉬운 구조와 성능 덕분에 우승모델(Googlenet)보다 인기가 더 많음
- 핵심
  - 모델 깊이와 성능의 관계 파악
    - 더 깊이 만들기 위해서 커널을 3x3을 고정
    - 큰 필터를 한번 적용하는것보다 작은 필터를 여러번 적용하는것이 성능은 같은데 파라미터 수가 더 적음
    - 작은 필터를 적용하니 비선형성이 증가하여 특성을 잘 나타냄

- input : 224 x 224 x 3

In [34]:
from torch.nn.modules.activation import Softmax # 마지막 이미지 분류시 사용
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda # ToTensor : y값을 텐서로 바꾸고 정규화
                                                    # Lambda : 

import os

In [2]:
device = 'cude' if torch.cuda.is_available() else 'cpu' 

In [33]:
# Retrieve data directly from Stanford data source
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

# Unzip raw zip file
!unzip -qq 'tiny-imagenet-200.zip'

# Define main data directory
DATA_DIR = 'tiny-imagenet-200'

--2022-12-17 06:28:13--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  56.4MB/s    in 5.1s    

2022-12-17 06:28:19 (46.7 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [40]:
# Define training and validation data paths
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

training_data = datasets.ImageFolder(root = TRAIN_DIR, transform=ToTensor())
test_data = datasets.ImageFolder(root = TEST_DIR, transform=ToTensor())

### 추가학습
- 직접 데이터셋을 구성하고 불러오는 방법
- https://data-panic.tistory.com/13

- 참조한 페이지(tiny-imagenet-200 관련)
- https://towardsdatascience.com/pytorch-ignite-classifying-tiny-imagenet-with-efficientnet-e5b1768e5e8f

In [32]:
from torch.nn.modules import dropout
from torch.nn.modules.adaptive import Linear
from torch.nn.modules.pooling import MaxPool2d
class VGG16(nn.Module):
  def __init__(self):
    super(VGG16, self).__init__()
    self.flatten = nn.Flatten()

    """Conv layers"""
    self.conv1_1 = nn.Sequential(
        nn.Conv2d(3, 64, 3, padding=1),
        nn.ReLU(),
    )
    self.conv1_2 = nn.Sequential(
        nn.Conv2d(64, 64, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.conv2_1 = nn.Sequential(
        nn.Conv2d(64, 128, 3, padding=1),
        nn.ReLU()
    )
    self.conv2_2 = nn.Sequential(
        nn.Conv2d(128, 128, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.conv3_1 = nn.Sequential(
        nn.Conv2d(128, 256, 3, padding=1),
        nn.ReLU()
    )
    self.conv3_2 = nn.Sequential(
        nn.Conv2d(256, 256, 3, padding=1),
        nn.ReLU()
    )
    self.conv3_3 = nn.Sequential(
        nn.Conv2d(256, 256, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.conv4_1 =  nn.Sequential(
        nn.Conv2d(256, 512, 3, padding=1),
        nn.ReLU()
    )
    self.conv4_2 =  nn.Sequential(
        nn.Conv2d(512, 512, 3, padding=1),
        nn.ReLU()
    )
    self.conv4_3 = nn.Sequential(
        nn.Conv2d(512, 512, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.conv5_1 = nn.Sequential(            # 반절 크기를 같은 필터수로 연산 진행
        nn.Conv2d(512, 512, 3, padding=1),
        nn.ReLU()
    )
    self.conv5_2 = nn.Sequential(            
        nn.Conv2d(512, 512, 3, padding=1),
        nn.ReLU()
    )
    self.conv5_3 = nn.Sequential(            
        nn.Conv2d(512, 512, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    """FC layers"""
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(7*7*512, 4096, dropout=0.4), # 논문에 나온대로 숫자 그대로 넣으면 편함
        nn.ReLU(),
        nn.Linear(4096, 1000),
        nn.Softmax(1000)
    )

RuntimeError: ignored